In [ ]:
import json
import os

import importlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import seaborn as sns
from sklearn import metrics

%matplotlib inline

# Get list of available train/test files for chosen run folder
run_folder_kcm = "../results/debug/"
run_folder_atb = "../results/debug/"
run_folder_tte = "../results/debug/"

In [ ]:
models = []
train_nets = []
test_nets = []
errors = []
results = kcm_kcm_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("KCM")
    test_nets.append("KCM")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))
results = kcm_atb_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("KCM")
    test_nets.append("ATB")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))
results = atb_atb_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("ATB")
    test_nets.append("ATB")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))
results = atb_kcm_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("ATB")
    test_nets.append("KCM")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))

metric_results = pd.DataFrame(columns=["Model","Train Network","Test Network","Metric"])
metric_results['Model'] = models
metric_results['Train Network'] = train_nets
metric_results['Test Network'] = test_nets
metric_results['Metric'] = errors
metric_results['Model-Train-Test'] = metric_results['Model']+"_"+metric_results['Train Network']+"_"+metric_results['Test Network']
metric_results = metric_results.sort_values('Model-Train-Test')
metric_results